In [30]:
import pandas as pd
import requests
import json
import os
import getpass
from pathlib import Path
import sqlite3
from time import sleep
import helpful_functions as hf
import ace_lib as ace
import pickle

In [31]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_FJP1D6MbJrD4NhHmKe3g4acXcnUg



In [32]:
dataset_id="fundamental28"
datasets_df = hf.get_datafields(s, region="GLB",dataset_id=dataset_id, universe="MINVOL1M", delay=1) # by default we load all datasets USA TOP3000 delay 1
datasets_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,userCount,alphaCount,pyramidMultiplier,themes
0,fnd28_a_value_08301,value of field: Return on Equity Total (%),"{'id': 'fundamental28', 'name': 'Global Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9509,57,106,2.0,[]
1,fnd28_a_value_08316,value of field: Operating Profit Margin,"{'id': 'fundamental28', 'name': 'Global Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9555,37,74,2.0,[]
2,fnd28_a_value_08326,value of field: Return on Assets,"{'id': 'fundamental28', 'name': 'Global Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9565,35,61,2.0,[]
3,fnd28_a_value_08346,value of field: Tax Rate,"{'id': 'fundamental28', 'name': 'Global Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9131,23,41,2.0,[]
4,fnd28_a_value_08366,value of field: Net Margin,"{'id': 'fundamental28', 'name': 'Global Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9543,63,158,2.0,[]


In [33]:
datafields_df=datasets_df

In [34]:
datafields_df_vector = datafields_df[datafields_df["type"] == "VECTOR"]
datafields_df_matrix = datafields_df[datafields_df["type"] == "MATRIX"]

# Sort datafields by alphaCount in descending order
datafields_df_matrix = datafields_df_matrix.sort_values(by="alphaCount", ascending=False)
datafields_df_vector = datafields_df_vector.sort_values(by="alphaCount", ascending=False)

# Display the length of each datafield type
print(len(datafields_df_vector))
print(len(datafields_df_matrix))

0
929


In [35]:

li = datafields_df_matrix.id.values.tolist()
len(li)

929

In [42]:
expressions = [
   "zscore(group_zscore(ts_backfill({}, 120), sector) - group_zscore(ts_backfill(fnd28_a_value_08301, 120), sector)) ",
    
]

In [42]:
expressions = [
   "zscore(group_zscore(ts_backfill({}, 120), sector) - group_zscore(ts_backfill(fnd28_a_value_08301, 120), sector)) ",
    
]

In [43]:
count = 0
for i in li:
    for expr in expressions:
        try:
            count = count + 1
            print(count)
            if(count<4) : continue;
            
            simulate_data = {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "MARKET",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            }
            
            simulate_response = s.post('https://api.worldquantbrain.com/simulations', json=simulate_data)
            print(simulate_data)
            print(simulate_response)
    
            
            simulation_progress_url = simulate_response.headers['Location']
            print(simulation_progress_url)
            finished = False
            
            while True:
                simulation_progress = s.get(simulation_progress_url)
              
                print('simulation_progress')
                if simulation_progress.headers.get("Retry-After", 0) == 0:
                    break
                sleep(float(simulation_progress.headers["Retry-After"]))
                
        except Exception as e:
            print(e)
        
 

1
2
3
4
{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'GLB', 'universe': 'MINVOL1M', 'delay': 1, 'decay': 0, 'neutralization': 'SECTOR', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'language': 'FASTEXPR', 'visualization': False, 'testPeriod': 'P2Y0M'}, 'regular': 'zscore(group_zscore(ts_backfill(fnd28_a_value_08316, 120), sector) - group_zscore(ts_backfill(fnd28_a_value_08301, 120), sector)) '}
<Response [201]>
https://api.worldquantbrain.com/simulations/3npeHRbuY56sabh8gdjyrBP
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simulation_progress
simu

KeyboardInterrupt: 

In [44]:
count = 0
for i in li:
    for expr in expressions:
        try:
            count = count + 1
            print(count)
            if(count<4) : continue;
            
            simulate_data = {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "MARKET",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            }
            
            simulate_response = s.post('https://api.worldquantbrain.com/simulations', json=simulate_data)
            print(simulate_data)
            print(simulate_response)
    
            
            simulation_progress_url = simulate_response.headers['Location']
            print(simulation_progress_url)
            finished = False
            
            while True:
                simulation_progress = s.get(simulation_progress_url)
              
                print('simulation_progress')
                if simulation_progress.headers.get("Retry-After", 0) == 0:
                    break
                sleep(float(simulation_progress.headers["Retry-After"]))
                
        except Exception as e:
            print(e)
        
 

NameError: name 'X' is not defined

In [ ]:
simulation_data=[
    {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "MARKET",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            },
    {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "SECTOR",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            },
    {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "SUBINDUSTRY",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            },
    {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "INDUSTRY",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            }
]